# Initialisation

In [745]:
import matplotlib
matplotlib.use('Qt5Agg')  # Utiliser le backend Qt5Agg pour windows
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import numpy as np
import os
from PIL import Image
import lmfit
#%matplotlib tk #pour Linux

# Importer fichier

In [746]:
path_to_tiff = os.path.join("..", "acquisition", "video_output_carac_150ms_1im_1um.tiff")

tiff = Image.open(path_to_tiff)

# Nombre de frames pas vide

In [747]:
with Image.open(path_to_tiff) as img:
    frame_number = 0
    actual_frames = 0
    try:
        while True:
            frame_number += 1
            if np.sum(np.array(img)) != 0:
                actual_frames += 1
                
            img.seek(frame_number)
    except EOFError:
        print("All frames processed.")

actual_frames

All frames processed.


71

# 1re frame

**Le nombre de frames ignorés n'est pas pris en compte**

In [748]:
frame_index = 0
tiff.seek(frame_index)
original_image = np.array(tiff)

while np.sum(original_image) == 0:
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    print(frame_index)

1


# Traitement d'image

In [749]:
clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
preprocessed = clahe.apply(original_image)

blurred = cv2.medianBlur(preprocessed, 115)
preprocessed2 = cv2.subtract(preprocessed, blurred)

# Apply Non-Local Means Denoising
img = cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Sélection du point à tracker

In [750]:
def crop(img, x, y, crop_size=100):
    x_start = int(x - crop_size // 2)
    x_end = int(x + crop_size // 2)
    y_start = int(y - crop_size // 2)
    y_end = int(y + crop_size // 2)

    return img[y_start:y_end, x_start:x_end]

In [751]:
# Display the image and let the user select a point interactively
fig, ax = plt.subplots()
ax.imshow(img, cmap='gray')  # Use 'gray' for better visibility of grayscale images
plt.title(f"Frame {frame_index}: Select a point")

# Ask for a point to be selected
print("Please click on the point you want to select.")
x, y = plt.ginput(1)[0]  # This will get the coordinates of the clicked point
print(f"Selected point: ({x}, {y})")
plt.close()

crop_sze = 100

Please click on the point you want to select.
Selected point: (444.82467532467535, 567.551948051948)


# Fit gaussien sur le point sélectionné

In [752]:
def prepare_data(x, y, z):
    return (x.flatten(), y.flatten()), z.flatten()

In [753]:
def gaussian_2d(xy, amplitude, x0, y0, sigma_x, sigma_y, offset):
    x, y = xy
    a = 1 / (2 * sigma_x**2)
    b = 1 / (2 * sigma_y**2)
    return offset + amplitude * np.exp(- (a * (x - x0)**2 + b * (y - y0)**2))

In [754]:
def localisateur_gaussien(intensity_grid, maxi):
    x = np.arange(intensity_grid.shape[0])
    y = np.arange(intensity_grid.shape[1])
    X, Y = np.meshgrid(x, y)

    # Préparer les données pour le fit
    (xdata, ydata), zdata = prepare_data(X, Y, intensity_grid)
    model = lmfit.Model(gaussian_2d)
    max_idx = np.unravel_index(np.argmax(intensity_grid), intensity_grid.shape)
    initial_x0 = x[max_idx[0]]
    initial_y0 = y[max_idx[1]]

    # Définir les paramètres du modèle
    params = model.make_params(
        amplitude=np.max(intensity_grid),
        x0=initial_x0,
        y0=initial_y0,
        sigma_x=1,
        sigma_y=1,
        offset=2
    )

    # Effectuer l'ajustement
    result = model.fit(zdata, params, xy=(xdata, ydata))

    x_position = result.params['x0'].value + maxi[0] - 24.5
    y_position = result.params['y0'].value + maxi[1] - 24.5

    return [x_position, y_position], result.params['sigma_x'].value, result.params['sigma_y'].value

# Process d'image (enlever le bruit)

**semble faire du trouble**

In [755]:
def denoise(image):
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(30, 30))
    preprocessed = clahe.apply(image)
    
    blurred = cv2.medianBlur(preprocessed, 115)
    preprocessed2 = cv2.subtract(preprocessed, blurred)
    
    return cv2.fastNlMeansDenoising(preprocessed2, None, 15, 7, 41)

# Passe au prochain frame

In [756]:
def next_frame(frame_index):
    frame_index += 1
    tiff.seek(frame_index)
    original_image = np.array(tiff)
    
    while np.sum(original_image) == 0:
        frame_index += 1
        tiff.seek(frame_index)
        original_image = np.array(tiff)

    return [frame_index, original_image]

# Faire le crop et fit

In [757]:
def particle_tracker(image, x, y, frame_index):
    image = denoise(image)
    
    cropped_img = crop(image, x, y, crop_sze)

    result_fit = localisateur_gaussien(cropped_img, [x, y])

    x_new, y_new = result_fit[0][0], result_fit[0][1]

    return [result_fit, cropped_img, (x_new, y_new), (result_fit[1], result_fit[2])]

# Main loop

In [758]:
position_list = []
sigma_list = []
crop_frames = []
big_frames = []

for _ in range(actual_frames - 1):
    data = particle_tracker(img, x, y, frame_index)
    position_list.append(data[2])
    sigma_list.append(data[3])
    frame_index, img = next_frame(frame_index)
    big_frames.append(img)
    crop_frames.append(data[1])

# Résultats

In [759]:
position_list

[(469.0140715166091, 595.0868553643782),
 (480.82235671223543, 594.5845544826493),
 (475.2956891049118, 608.2464137446578),
 (521.1620747184403, 555.3073435348357),
 (495.8247153943396, 598.253450203672),
 (460.0722652340559, 557.1142662587856),
 (489.83535775283167, 599.6192369933627),
 (506.35314497631794, 589.4130141545298),
 (508.0984642197227, 595.0995937905839),
 (504.8247036132108, 593.1375920610816),
 (-9355.56051589632, 568.8328120150023),
 (16623.60327839273, 575.1043481657723),
 (510.6821191370706, 499.6774508098857),
 (-379550.2063976411, 575.3142529921339),
 (-10789.346675034287, 572.002082451998),
 (507.8959294881398, 556.4693473142535),
 (-18714.47190801821, 575.5697584676012),
 (-10046.963832762394, 570.2450627006979),
 (-3526.334235104622, 572.75517800261),
 (-19778.073740331783, 564.4881543140384),
 (-331.910753758655, 571.5812135049955),
 (505.62844719620057, 583.5574863038631),
 (504.25057210221473, 607.85209372356),
 (215.61263442910982, 571.1263722472389),
 (-406.

In [760]:
sigma_list

[(3.221087668458898, 4.364936315463081),
 (-0.37551779966706206, 0.005035136394737197),
 (2.974432646380411, 4.602829806653173),
 (260.18055447169803, -24.577981286351093),
 (-0.5356210502106258, 0.10910452751141551),
 (72.92969469945378, 25.487275106475067),
 (0.30888460432573495, 1.203496940422119),
 (9.658258770523203, 45.35782351795441),
 (-10.16878509468356, -73.30705452097482),
 (-0.3266267237278279, -0.15628428454663676),
 (2535.8387880090527, 50.631146812525294),
 (-13835.55803514551, 40.964690649775925),
 (9.841359844244812, -86.79527534684364),
 (-1190785.625138332, -4041.337240414852),
 (6204.0337816541805, 50.70762778848079),
 (9.790400336430206, 80.38134835875792),
 (8678.843772664888, 81.50081224783915),
 (2551.7998491784874, 38.707522601120296),
 (2342.300184567241, 55.70123593126429),
 (10186.247212591812, 101.4428520590631),
 (570.2264546909885, 32.77145799139029),
 (-11.884883260092316, -42.432471162441686),
 (-8.43666256986014, -9.758294830815837),
 (727.649776599414

In [761]:
x_plt, y_plt = zip(*position_list)
# Create the plot
plt.figure(figsize=(10, 8))  # Optional: Adjust the figure size
plt.plot(x_plt, y_plt, marker='o', linestyle='-', color='b', label='Connected Points')

# Set grid limits to match the 1440x1080 grid
plt.xlim(0, 1440)
plt.ylim(0, 1080)

# Add labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('2D Connected Points Plot')
plt.legend()

# Optional: Add grid lines
plt.grid(True)

## anim toutes les particules

In [762]:
# fig, ax = plt.subplots()
# img = ax.imshow(big_frames[0], cmap='gray', animated=True)


# # Update function
# def update(frame):
#     img.set_array(frame)
#     return img,
    
# ani = animation.FuncAnimation(fig, update, frames=big_frames, interval=50, blit=True)
# plt.show()

## anim crop

In [763]:
fig, ax = plt.subplots()
img = ax.imshow(crop_frames[0], cmap='gray', animated=True)


def update(frame):
    img.set_array(frame)
    return img,
    
ani = animation.FuncAnimation(fig, update, frames=crop_frames, interval=50, blit=True)
plt.show()